#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

<b> 1. 동작에 필요한 패키지 설치 </b>

In [ ]:
%pip install python-dotenv openai faiss-cpu langchain pypdf tiktoken
%pip install -U langchain-openai

<b> 2. LLM 연결 API Key 설정 </b>

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

<b> 3. PDF 파일에서 텍스트 추출 </b>

In [4]:

from pypdf import PdfReader

def load_pdf_text(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# 사용 예
pdf_path = "./attention.pdf"  # 같은 폴더에 PDF가 있어야 함
raw_text = load_pdf_text(pdf_path)
print(raw_text[:1000])  # 처음 1000자만 출력


Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Exp

<b> 4. 텍스트 분할 </b>

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def split_text(text, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.create_documents([text])

# 사용 예
documents = split_text(raw_text)
print(f"{len(documents)}개 문단 생성됨")
print(documents[0])


89개 문단 생성됨
page_content='Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu'


<b> 5. 벡터 임베딩 및 벡터 DB (FAISS) 인덱싱 </b>

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings()

# 벡터 DB 생성
vector_db = FAISS.from_documents(documents, embedding_model)


<b> 6. 사용자 쿼리에 대한 검색 </b>

In [7]:
def retrieve_docs(query, db, k=3):
    return db.similarity_search(query, k=k)

# 예시 쿼리
query = "이 논문에서 제안한 Attention과 Self-Atteion은 무엇인가요?"
matched_docs = retrieve_docs(query, vector_db)
for i, doc in enumerate(matched_docs):
    print(f"[{i+1}] {doc.page_content[:300]}...\n")


[1] reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism rela...

[2] Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms
a...

[3] The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Figure 5: Many of the attention heads exhibit ...



<b> 7. LLM으로 답변 생성 </b>

In [8]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

def generate_answer(docs, user_query):
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = f"""다음은 논문에서 발췌한 내용입니다:\n\n{context}\n\n사용자 질문: {user_query}\n\n논문 내용을 참고하여 질문에 답변해 주세요."""
    
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content

# 사용자 입력 받기
# query = input("질문을 입력하세요: ")
query = "Attention과 Self-Attention을 비교하시오."

# 답변 생성
answer = generate_answer(matched_docs, query)
print("\n💡 답변:\n")
print(answer)



💡 답변:

Attention과 Self-Attention은 모두 시퀀스 모델링에서 중요한 역할을 하는 메커니즘이지만, 그 목적과 적용 방식에서 차이가 있습니다.

1. **Attention**:
   - Attention 메커니즘은 입력 또는 출력 시퀀스 내의 서로 다른 위치 간의 의존성을 모델링하는 데 사용됩니다. 이는 입력 시퀀스의 특정 부분에 더 많은 가중치를 부여하여 모델이 중요한 정보에 집중할 수 있도록 합니다.
   - Attention은 주로 시퀀스-투-시퀀스 모델에서 사용되며, 특히 입력과 출력 시퀀스 간의 관계를 모델링하는 데 유용합니다.
   - 전통적으로 Attention은 RNN과 같은 순환 신경망과 함께 사용되어 왔습니다. 이는 시퀀스의 길이에 관계없이 의존성을 모델링할 수 있도록 합니다.

2. **Self-Attention**:
   - Self-Attention, 또는 Intra-Attention은 단일 시퀀스 내의 서로 다른 위치 간의 관계를 모델링하는 메커니즘입니다. 이는 시퀀스의 각 위치가 다른 모든 위치와의 관계를 고려하여 시퀀스의 표현을 계산하는 데 사용됩니다.
   - Self-Attention은 Transformer와 같은 모델에서 핵심적인 역할을 하며, 순환 구조 없이도 시퀀스 내의 모든 위치 간의 의존성을 효과적으로 모델링할 수 있습니다.
   - Self-Attention은 특히 긴 시퀀스에서도 병렬 처리가 가능하다는 장점이 있어, 계산 효율성을 크게 향상시킵니다.

논문에서는 Transformer 모델이 순환 구조를 배제하고 Self-Attention 메커니즘을 사용하여 시퀀스 내의 의존성을 모델링한다고 설명하고 있습니다. 이는 Attention 메커니즘이 RNN과 함께 사용되는 기존 방식과 대비됩니다. Self-Attention은 여러 헤드를 사용하여 다양한 관점에서 시퀀스를 분석할 수 있도록 하며, 이는 모델의 표현력을 높이는 데 기여합니다.
